In [ ]:
import sys
import numpy as np
import subprocess
import time
import os

import getComps
import makeStdCmp
import SCALEDepleter.depletion_python_scripts.runAndKillScale as runAndKillScale
import makeTritonFile
import copyMatAndF33Files
import makeAndRunOrigen
import removeAndMakeDir
# some notes:
# FMA=fission matrix
#
#
#
##################################################################
########################### USER INPUT ###########################
##################################################################
"""
  Step 0 - ask yourself:
  Is first step length ok in the input file?
  Is the FMA parameter set to yes?
  Are the NPG and other neutron parameters correct?
"""

# fissionable regions - used for origen later
fissionable_mats = [101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116]
fissionable_mats_vols = [6.02834870915574000000E+04]*16
residual_number_density = 1e-20

# add addnux dict location

# TODO add volumes, get fission source shape after monte carlo from MC output, use fission source shape to adjust poewrs in each node.

# ORIGEN information
specific_power = [1e-5]
steplength_days = [5]
origen_predictor_divs = 100 # number of time divs for predictor
origen_corrector_divs = 100 # number of time divs for corrector

# file handles
addnuxdictbase = 'addnuxDicts/addnux1Dict.dict'
base_triton = 'triton_base.inp'
origen_base = 'baseOrigenFile.inp'
blender_base = 'blenderBase.inp'
origenResults_F71dir = 'OrigenResults_F71dir'
MonteCarloResults_F33dir = 'MonteCarloResults_F33'
# nprocs
Nprocs = 48


In [2]:

##################################################################
############################ SETUP ###############################
##################################################################

# get temp dir for transport calculations
# tmpdir = sys.argv[2]
tmpdir = 'tmp'


# get machine file
# machinefile = sys.argv[1]
machinefile = 'kasdakljsdasda'

# scale run handles
print("NPROCS", Nprocs)
print("MACHINEFILE",machinefile)
print("BASE_TRITON",base_triton)



NPROCS 48
MACHINEFILE kasdakljsdasda
BASE_TRITON triton_base.inp


In [3]:
# Get materials at time 0 and append
time_lib = getComps.time_dependent_material_lib()
fh = open(base_triton, 'r')
initial_mats = getComps.get_comps(fh)
fh.close()
time_lib.append_lib(initial_mats, time=0.0, step=int(0), PC_flag='C') # C flag since it is the "true" solution

# completely remove and remake some directories - since we are makign a new case
removeAndMakeDir.removeAndMakeDir(dirct=tmpdir)
removeAndMakeDir.removeAndMakeDir(dirct=MonteCarloResults_F33dir)
removeAndMakeDir.removeAndMakeDir(dirct=origenResults_F71dir)

# we need to add initial nuclides to addnuxdict - addnux dict holds all the addnux nuclides + nuclides defined in the initial material defs
addnuxdict, new_isotopes_from_addnux = getComps.makeNewAddnuxDict(time_lib.mats_by_steps[0]['C'].material_dict, tmpdir, addnuxdictbase)

# any isotopes that are in the addnux dict but not the initial comp now needs to be added as residual isotopes
for iso in new_isotopes_from_addnux:
  mat_lib = time_lib.mats_by_steps[0]['C']
  mat_lib.add_residual_isotope(isotope=iso, numdens=residual_number_density)

# make a base triton scale input with flags for stdcmp - make for predicted and corrected stdcmps
corrected_triton_input = makeTritonFile.makeTritonFile(base_triton, fissionable_mats, stdcmp_tag='corrected_cmp') # uses "corrcted" stdcmp material defs
predicted_triton_input = makeTritonFile.makeTritonFile(base_triton, fissionable_mats, stdcmp_tag='predicted_cmp') # uses "predicted" stdcmp material defs

# make a stdcmp file based on initial materials - automatically move stdcmps for triton_stdcmp to tmp dir
for material_index in fissionable_mats:
  thisfilename = 'StdCmpMix'+str(material_index)+'_corrected_cmp'
  makeStdCmp.makeStdCmpFromMatLib(outputFilename=thisfilename, material_lib=initial_mats, material_index=material_index, tmpdir=tmpdir)

# scale_run_line = ['scalerte', '-N', str(Nprocs), '-M',machinefile, base_triton_stdcmp, '-T', tmpdir, '-m'] # for cluster.
scale_run_line_c = ['scalerte', corrected_triton_input, '-T', tmpdir, '-m'] # for pc
scale_run_line_p = ['scalerte', predicted_triton_input, '-T', tmpdir, '-m'] # for pc
# start iteration
step_num = 0 # temporary thing

### FOR LOOP STARTS HERE

# set origen temp dir


read comp found! Now starting comp directory.
end comp found! Now finalizing comp directory.
read comp found! Now starting comp directory.
end comp found! Now finalizing comp directory.
read comp found! Now starting comp directory.
end comp found! Now finalizing comp directory.
Now making standard comp file for material 101  | output= StdCmpMix101_corrected_cmp
Now making standard comp file for material 102  | output= StdCmpMix102_corrected_cmp
Now making standard comp file for material 103  | output= StdCmpMix103_corrected_cmp
Now making standard comp file for material 104  | output= StdCmpMix104_corrected_cmp
Now making standard comp file for material 105  | output= StdCmpMix105_corrected_cmp
Now making standard comp file for material 106  | output= StdCmpMix106_corrected_cmp
Now making standard comp file for material 107  | output= StdCmpMix107_corrected_cmp
Now making standard comp file for material 108  | output= StdCmpMix108_corrected_cmp
Now making standard comp file for materia

In [4]:

################################################
################## PREDICTOR ###################
################################################

# STEP 1 - run transport with base file and then copy files from temp dir
keff_line = runAndKillScale.runAndKillScale(scale_run_line_c, corrected_triton_input)

# make a folder with all f33's from the tmpdir moved into the new folder - label folder with step number - f33_files is a dict[material_id]
f33_files = copyMatAndF33Files.copy_files_from_temp(tmpdir=tmpdir, step_num=step_num, mcf33dir=MonteCarloResults_F33dir)


Started process with PID: 459821
Now sleeping python for 5 seconds
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                        

mkdir: cannot create directory ‘MonteCarloResults_F33’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step0’: File exists
mkdir: cannot create directory ‘MonteCarloRe

In [ ]:
# Predictor step of origen - first write origen file for this step.
origen_file_list = []
origen_tmpdirs = []


# make origen file, append origen file to list of origen files, append Beginning of step mat composition to the material library.
for fiss_mat_id in fissionable_mats:
  bos_mat_lib = time_lib.material_lib_from_step(requested_step=step_num, PC_flag='C')
  bos_mat_lib_this_mat = bos_mat_lib.material_dict[fiss_mat_id]
  origen_file_handle, origen_tmpdir = makeAndRunOrigen.makeOrigenFile(origen_base=origen_base, fiss_mat_id=fiss_mat_id, f33_files=f33_files,
                                                                      origenResults_F71dir=origenResults_F71dir, step_num=step_num,
                                                                      steplength_days=steplength_days, origen_predictor_divs=origen_predictor_divs,
                                                                      specific_power=specific_power, predictor_corrector_string='PREDICTOR',
                                                                      bos_cmp=bos_mat_lib_this_mat)
  origen_file_list.append(origen_file_handle)
  origen_tmpdirs.append(origen_tmpdir)


# run origen now, unpack isotopics, then append isotopics to time_lib
for idx, fiss_mat_id in enumerate(fissionable_mats):
  # get temp directory for this origen directory
  temp = origen_tmpdirs[idx]
  # get origen input
  origen_inp = origen_file_list[idx]
  # run origen
  origen_output_loc = makeAndRunOrigen.runOrigenFile(tmpdir=temp, origen_file=origen_inp, material_id=fiss_mat_id, skipRunning=False)
  origen_f71_loc = origenResults_F71dir+'/PREDICTOR_EOS_step'+str(step_num)+'_mat'+str(fiss_mat_id)+'.f71'

  # now unpack isotopes from output using output location
  stdCmpFilename = 'StdCmpMix'+str(fiss_mat_id)+'_predicted_cmp'
  temperature = time_lib.material_lib_from_step(requested_step=step_num, PC_flag='C').material_dict[fiss_mat_id].temp # temperature from beginning of simulation for this matid

  # make a predicted stdcmp from the f71 file at the end of the predictor step
  _ = makeStdCmp.makeStdCmpFromF71(materialNumber=fiss_mat_id, temperature=temperature, filename=origen_f71_loc, dictionaryFilename=addnuxdict, outputFilename=stdCmpFilename, tmpdir=tmpdir)

  # eos_mat_lib = makeStdCmp.makeMatLibFromF71(filename=origen_f71_loc) #



 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_PREDICTO

cp $INPDIR/../MonteCarloResults_F33/step0/101.f33 this_f33.f33
  Now executing origen                          
Scale job /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_origen_PREDICTOR_step0/ORIGEN_PREDICTOR_step0_mat101.inp is finished.
Output is stored in /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_origen_PREDICTOR_step0/ORIGEN_PREDICTOR_step0_mat101.out
Now making standard comp file for material 101  | output= StdCmpMix101_predicted_cmp
                     NOW RUNNING F71 CONVERSIONS 
STARTING LINE
am-241 101 0 2.96358042E-16 900 end
am-242 101 0 7.27008447E-19 900 end
am-243 101 0 1.72148399E-18 900 end
be-9 101 0 1.93693772E-03 900 end
c-12 101 0 7.48983244E-02 900 end
cm-242 101 0 5.34859862E-19 900 end
cm-243 101 0 9.90849640E-21 900 end
f-19 101 0 7.74773791E-03 900 end
li-6 101 0 2.23252702E-07 900 end
li-7 101 0 3.87364645E-03 900 end
np-237 101 0 4.29328660E-12 900 end
o-16 101 0 2.70206057E-04 900 end
pu-238 101 0 2.38603850E-15 900 end
pu-239 101 0 3

rm: cannot remove 'StdCmpMix101_predicted_cmp': No such file or directory


 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

...

rm: cannot remove 'StdCmpMix114_predicted_cmp': No such file or directory


 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix115_predicted_cmp': No such file or directory


 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix116_predicted_cmp': No such file or directory


In [6]:

# now, we have to run eigenvalue calc after depleting, then the predictor step will be completed.
keff_line = runAndKillScale.runAndKillScale(scale_run_line_p, predicted_triton_input) # run scale for the 'predicted' triton input

# now copy over the new f33 files
f33_files = copyMatAndF33Files.copy_files_from_temp(tmpdir=tmpdir, step_num=step_num+1, mcf33dir=MonteCarloResults_F33dir)


Started process with PID: 460910
Now sleeping python for 5 seconds
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                        

mkdir: cannot create directory ‘MonteCarloResults_F33’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloResults_F33/step1’: File exists
mkdir: cannot create directory ‘MonteCarloRe

In [7]:
# now deplete using origen with the new f33 files from step_num+1

origen_file_list = []
origen_tmpdirs = []

# make origen file, append origen file to list of origen files, append Beginning of step mat composition to the material library.
for fiss_mat_id in fissionable_mats:
  bos_mat_lib = time_lib.material_lib_from_step(requested_step=step_num, PC_flag='C')
  bos_mat_lib_this_mat = bos_mat_lib.material_dict[fiss_mat_id]
  origen_file_handle, origen_tmpdir = makeAndRunOrigen.makeOrigenFile(origen_base=origen_base, fiss_mat_id=fiss_mat_id, f33_files=f33_files,
                                                                      origenResults_F71dir=origenResults_F71dir, step_num=step_num,
                                                                      steplength_days=steplength_days, origen_predictor_divs=origen_predictor_divs,
                                                                      specific_power=specific_power, predictor_corrector_string='CORRECTOR',
                                                                      bos_cmp=bos_mat_lib_this_mat)
  origen_file_list.append(origen_file_handle)
  origen_tmpdirs.append(origen_tmpdir)


# run origen now, unpack isotopics, then append isotopics to time_lib
for idx, fiss_mat_id in enumerate(fissionable_mats):
  # get temp directory for this origen directory
  temp = origen_tmpdirs[idx]
  # get origen input
  origen_inp = origen_file_list[idx]
  # run origen
  origen_output_loc = makeAndRunOrigen.runOrigenFile(tmpdir=temp, origen_file=origen_inp, material_id=fiss_mat_id, skipRunning=False)
  origen_f71_loc = origenResults_F71dir+'/CORRECTOR_EOS_step'+str(step_num)+'_mat'+str(fiss_mat_id)+'.f71'






 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTOR_step0’: File exists
mkdir: cannot create directory ‘tmp_origen_CORRECTO

cp $INPDIR/../MonteCarloResults_F33/step1/101.f33 this_f33.f33
  Now executing origen                          
Scale job /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_origen_CORRECTOR_step0/ORIGEN_CORRECTOR_step0_mat101.inp is finished.
Output is stored in /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_origen_CORRECTOR_step0/ORIGEN_CORRECTOR_step0_mat101.out
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and

In [41]:
# now blend files together and calculate the average of the predictor and corrector nuclide vector using Origen - predictor and corrector blended - 0.5

corrected_mat_lib = getComps.material_lib()
for idx, fiss_mat_id in enumerate(fissionable_mats):
  blender_tmp, new_input_name, path_to_f71 = makeAndRunOrigen.origenBlend(origen_f71_results_dir=origenResults_F71dir, step_num=step_num, mat_id=fiss_mat_id, blended_basefilename=blender_base)
  makeAndRunOrigen.runOrigenFile(tmpdir=blender_tmp, origen_file=new_input_name, material_id=-1, skipRunning=False)

  # make a standard cmp from the f71 - these are the corrected isotopes after averaging
  stdCmpFilename = 'StdCmpMix'+str(fiss_mat_id)+'_corrected_cmp'
  temperature = time_lib.material_lib_from_step(requested_step=0, PC_flag='C').material_dict[fiss_mat_id].temp # temperature from beginning of simulation for this matid
  origen_f71_loc = path_to_f71

  # make corrector stdcmp and place them in tmpdir
  _ = makeStdCmp.makeStdCmpFromF71(materialNumber=fiss_mat_id, temperature=temperature, filename=origen_f71_loc, dictionaryFilename=addnuxdict, outputFilename=stdCmpFilename, tmpdir=tmpdir)

  # get material
  corrected_mat_lib.append_mat_to_lib(getComps.get_comps_from_std_mix_file(tmpdir+'/'+stdCmpFilename))

# append corrected mat lib to time lib
time_lib.append_lib(time=5, lib=corrected_mat_lib, step=step_num+1, PC_flag='C')



 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix101_corrected_cmp': No such file or directory


np-237 101 0 4.19407261E-12 900 end
o-16 101 0 2.70206031E-04 900 end
pu-238 101 0 2.32537278E-15 900 end
pu-239 101 0 3.58588916E-08 900 end
pu-240 101 0 2.63226062E-10 900 end
pu-241 101 0 2.08273257E-12 900 end
pu-242 101 0 4.23803961E-15 900 end
u-234 101 0 8.39686361E-13 900 end
u-235 101 0 1.70814874E-05 900 end
u-236 101 0 2.87445346E-08 900 end
u-238 101 0 1.72086317E-04 900 end
ENDING LINE
 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclea

mkdir: cannot create directory ‘tmp_blender’: File exists


cp $INPDIR/../OrigenResults_F71dir/CORRECTOR_EOS_step0_mat102.f71 c.f71
cp $INPDIR/../OrigenResults_F71dir/PREDICTOR_EOS_step0_mat102.f71 p.f71
  Now executing origen                          
cp $INPDIR/step0_mat102.f71 $INPDIR/../OrigenResults_F71dir/BLENDED_EOS_step0_mat102.f71
Scale job /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_blender/blenderBase_step0_mat102.inp is finished.
Output is stored in /home/jfaulkner31/projects/SCALETools/SCALEDepleter/tmp_blender/blenderBase_step0_mat102.out
Now making standard comp file for material 102  | output= StdCmpMix102_corrected_cmp
                     NOW RUNNING F71 CONVERSIONS 
STARTING LINE
am-241 102 0 3.86374959E-16 900 end
am-242 102 0 9.68258748E-19 900 end
am-243 102 0 2.49632139E-18 900 end
be-9 102 0 1.93693771E-03 900 end
c-12 102 0 7.48983243E-02 900 end
cm-242 102 0 7.08692338E-19 900 end
cm-243 102 0 9.92673398E-21 900 end
f-19 102 0 7.74773783E-03 900 end
li-6 102 0 2.23304618E-07 900 end
li-7 102 0 3.87364644E-0

...

rm: cannot remove 'StdCmpMix114_corrected_cmp': No such file or directory
mkdir: cannot create directory ‘tmp_blender’: File exists


 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix115_corrected_cmp': No such file or directory
mkdir: cannot create directory ‘tmp_blender’: File exists


 *******************************************************************************
 *                                                                             *
 *                                 SCALE 6.3.1                                 *
 *                                -------------                                *
 *                                   2023-02                                   *
 *                                                                             *
 *           SCALE:  A Comprehensive Modeling and Simulation Suite             *
 *                   for Nuclear Safety Analysis and Design                    *
 *                                                                             *
 *                      Reactor and Nuclear Systems Division                   *
 *                        Oak Ridge National Laboratory                        *
 *                                                                             *
 *                          

rm: cannot remove 'StdCmpMix116_corrected_cmp': No such file or directory


In [ ]:

# remove some random leftover files from origen:
makeAndRunOrigen.removePattern('F33*.f33')
makeAndRunOrigen.removePattern('ORIGEN_*.f33')
makeAndRunOrigen.removePattern('*.f71')
makeAndRunOrigen.removePattern('ORIGEN_*.msg')


In [ ]:
# debugging only -
makeAndRunOrigen.removePattern('ORIGEN_PREDICTOR_step0_mat*.inp')
makeAndRunOrigen.removePattern('STD_CMP_*')
makeAndRunOrigen.removePattern('triton*msg')
makeAndRunOrigen.removePattern('triton*path')
makeAndRunOrigen.removePattern('triton*out')
makeAndRunOrigen.removePattern('tmp*')
makeAndRunOrigen.removePattern('StdCmpMix1*_')

In [38]:
import importlib
importlib.reload(makeAndRunOrigen)
importlib.reload(getComps)

<module 'getComps' from '/home/jfaulkner31/projects/SCALETools/SCALEDepleter/getComps.py'>

In [ ]:
import powerFromOutput
import importlib
importlib.reload(makeAndRunOrigen)
importlib.reload(getComps)
importlib.reload(powerFromOutput)
power_by_step = {}
powerFromOutput.getPower(fission_mat_ids=fissionable_mats, include_non_fission_material_power=False, filename='triton_base_stdcmp_predicted_cmp.out')

{101: 0.036657654951436865,
 102: 0.04649544407730049,
 103: 0.057960348452988884,
 104: 0.06612095724441774,
 105: 0.07270952237909281,
 106: 0.07685991789326124,
 107: 0.07510763993191148,
 108: 0.07397616901972563,
 109: 0.07338039451286672,
 110: 0.07612896765795533,
 111: 0.07596875938720336,
 112: 0.07154300590767998,
 113: 0.06567037148292781,
 114: 0.056032842695504154,
 115: 0.04462300991288675,
 116: 0.03076499449284069}

In [1]:
# Open the pickled data:
import pickle

with open('keff.pkl', 'rb') as file:
  keff_lines = pickle.load(file)
with open('power.pkl', 'rb') as file:
  power_by_step = pickle.load(file)
with open('nuclides.pkl', 'rb') as file:
  nuclides = pickle.load(file)

In [4]:
keff_lines[0].split()

['best', 'estimate', 'system', 'k-eff', '1.23338', '+', 'or', '-', '0.00014']